# Plate eigenmode with dolfin

this is the same approach for the same topic but using dolfin instead of dolfinx

This needs to be run using the project:shells image

In [1]:
from dolfin import *
import dolfin
import matplotlib.pyplot as plt
import numpy as np

In [2]:

# --------------------
# Functions and classes
# --------------------
# Strain function
def epsilon(u):
    return 0.5*(dolfin.nabla_grad(u) + dolfin.nabla_grad(u).T)

# Stress function
def sigma(u):
    return lmbda*dolfin.div(u)*dolfin.Identity(3) + 2*mu*epsilon(u)


In [3]:

# --------------------
# Parameters
# --------------------
# Young modulus, poisson number and density
E, nu = 70.0E9, 0.23
rho = 2500.0

# Lame's constants
mu = E/2./(1+nu)
lmbda = E*nu/(1+nu)/(1-2*nu)

l_x, l_y, l_z = 1.0, 1.0, 0.01  # Domain dimensions
n_x, n_y, n_z = 20, 20, 2  # Number of elements


In [4]:
# --------------------
# Geometry
# --------------------
mesh = dolfin.BoxMesh(dolfin.Point(0.0, 0.0, 0.0), dolfin.Point(l_x, l_y, l_z), n_x, n_y, n_z)


In [5]:
# --------------------
# Function spaces
# --------------------
V = dolfin.VectorFunctionSpace(mesh, "CG", 2)
u_tr = dolfin.TrialFunction(V)
u_test = dolfin.TestFunction(V)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [6]:

# --------------------
# Forms & matrices
# --------------------
a_form = dolfin.inner(sigma(u_tr), epsilon(u_test))*dolfin.dx
m_form = rho*dolfin.inner(u_tr, u_test)*dolfin.dx



In [7]:
A = dolfin.PETScMatrix()
M = dolfin.PETScMatrix()
A = dolfin.assemble(a_form, tensor=A)
M = dolfin.assemble(m_form, tensor=M)

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


In [12]:
A_mat = A.mat()
M_mat = M.mat()

In [13]:
ki, kj, kv = A_mat.getValuesCSR()
Mi,Mj,Mv = M_mat.getValuesCSR() 

In [14]:
# kmat_numpy = kmat_numpy.getDenseArray()
import scipy.linalg as splg
from scipy.sparse import linalg
import scipy.sparse as spsparse

In [15]:
Ksparse = spsparse.csr_matrix((kv,kj,ki))
Msparse = spsparse.csr_matrix((Mv,Mj,Mi))

In [20]:
# check if symmetric
print((np.abs(Ksparse - Ksparse.T)>1e-10).nnz == 0)
print((np.abs(Msparse - Msparse.T)>1e-10).nnz == 0)

False
True


In [ ]:
%time w,v = spsparse.linalg.eigs(Ksparse,15,Msparse,which='SM')

In [23]:
w

array([  1.15648940e+14+0.j,   1.15648940e+14+0.j,   1.11477255e+14+0.j,
         1.11477255e+14+0.j,   1.09768241e+14+0.j,   1.09768241e+14+0.j,
         1.09219833e+14+0.j,   1.09219833e+14+0.j,   1.08627738e+14+0.j,
         1.08627950e+14+0.j,   1.08487504e+14+0.j,   1.08487245e+14+0.j,
         1.08453609e+14+0.j,   1.08458344e+14+0.j,   1.08458226e+14+0.j])

In [8]:
# --------------------
# Eigen-solver
# --------------------
# dolfin.PETScOptions.set("eps_view")
eigensolver = dolfin.SLEPcEigenSolver(A, M)

In [9]:
eigensolver.parameters["problem_type"] = "gen_hermitian"
eigensolver.parameters["spectrum"] = "smallest real"
eigensolver.parameters["spectral_transform"] = "shift-and-invert"
eigensolver.parameters["spectral_shift"] = 100.0

In [10]:
N_eig = 12   # number of eigenvalues
%time eigensolver.solve(N_eig)

In [11]:
eigensolver.get_number_converged()

0

In [12]:
Msparse[15,12]

NameError: name 'Msparse' is not defined

In [13]:
# Eigenfrequencies
for i in range(0, N_eig):
    # Get i-th eigenvalue and eigenvector
    # r - real part of eigenvalue
    # c - imaginary part of eigenvalue
    # rx - real part of eigenvector
    # cx - imaginary part of eigenvector
    r, c, rx, cx = eigensolver.get_eigenpair(i)

    # Calculation of eigenfrequency from real part of eigenvalue
    freq_3D = fe.sqrt(r)/2/fe.pi
    print("Eigenfrequency: {0:8.5f} [Hz]".format(freq_3D))

    # # Initialize function and assign eigenvector
    # eigenmode = fe.Function(V, name="Eigenvector " + str(i))
    # eigenmode.vector()[:] = rx


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to extract eigenpair from SLEPc eigenvalue solver.
*** Reason:  Requested eigenpair (0) has not been computed.
*** Where:   This error was encountered inside SLEPcEigenSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  74d7efe1e84d65e9433fd96c50f1d278fa3e3f3f
*** -------------------------------------------------------------------------
